## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

In [27]:
# Import API key
from config import g_key

In [30]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,61.00,78,100,10.00,overcast clouds
1,1,Dingle,PH,10.9995,122.6711,80.74,71,6,14.47,clear sky
2,2,Busselton,AU,-33.6500,115.3333,79.00,57,100,3.00,overcast clouds
3,3,Hofn,IS,64.2539,-15.2082,38.37,84,100,27.60,light rain
4,4,Melville,US,40.7934,-73.4151,53.60,32,1,6.91,clear sky


In [31]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Dingle,PH,10.9995,122.6711,80.74,71,6,14.47,clear sky
2,2,Busselton,AU,-33.6500,115.3333,79.00,57,100,3.00,overcast clouds
6,6,Butaritari,KI,3.0707,172.7902,81.25,75,100,8.86,light rain
8,8,Atuona,PF,-9.8000,-139.0333,79.52,75,34,16.75,scattered clouds
10,10,Amahai,ID,-3.3333,128.9167,86.20,59,16,0.89,few clouds
11,11,Merauke,ID,-8.4667,140.3333,78.60,87,98,6.11,overcast clouds
12,12,Kavieng,PG,-2.5744,150.7967,84.56,69,85,12.53,overcast clouds
19,19,Avarua,CK,-21.2078,-159.7750,80.60,83,75,4.61,light rain
22,22,Georgetown,MY,5.4112,100.3354,80.60,83,20,2.30,few clouds
23,23,Kununurra,AU,-15.7667,128.7333,86.00,79,40,2.30,scattered clouds


In [34]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                200
City                   200
Country                200
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [35]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [37]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dingle,PH,80.74,clear sky,10.9995,122.6711,
2,Busselton,AU,79.00,overcast clouds,-33.6500,115.3333,
6,Butaritari,KI,81.25,light rain,3.0707,172.7902,
8,Atuona,PF,79.52,scattered clouds,-9.8000,-139.0333,
10,Amahai,ID,86.20,few clouds,-3.3333,128.9167,
11,Merauke,ID,78.60,overcast clouds,-8.4667,140.3333,
12,Kavieng,PG,84.56,overcast clouds,-2.5744,150.7967,
19,Avarua,CK,80.60,light rain,-21.2078,-159.7750,
22,Georgetown,MY,80.60,few clouds,5.4112,100.3354,
23,Kununurra,AU,86.00,scattered clouds,-15.7667,128.7333,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dingle,PH,80.74,clear sky,10.9995,122.6711,SEGAYA PROPERTY
2,Busselton,AU,79.00,overcast clouds,-33.6500,115.3333,Observatory Guest House
6,Butaritari,KI,81.25,light rain,3.0707,172.7902,Isles Sunset Lodge
8,Atuona,PF,79.52,scattered clouds,-9.8000,-139.0333,Villa Enata
10,Amahai,ID,86.20,few clouds,-3.3333,128.9167,HOTEL ISABELA
11,Merauke,ID,78.60,overcast clouds,-8.4667,140.3333,PC
12,Kavieng,PG,84.56,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
19,Avarua,CK,80.60,light rain,-21.2078,-159.7750,Paradise Inn
22,Georgetown,MY,80.60,few clouds,5.4112,100.3354,Cititel Penang
23,Kununurra,AU,86.00,scattered clouds,-15.7667,128.7333,Hotel Kununurra


In [40]:
# Determine if there are any empty rows.
hotel_df.count()

City                   200
Country                200
Max Temp               200
Current Description    200
Lat                    200
Lng                    200
Hotel Name             200
dtype: int64

In [41]:
# 7. Drop the rows where there is no Hotel Name.


In [43]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")


In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[["Lat", "Lng"]]
locations_df

,Lat,Lng
1,10.9995,122.6711
2,-33.6500,115.3333
6,3.0707,172.7902
8,-9.8000,-139.0333
10,-3.3333,128.9167
...,...,...
668,4.7485,-6.6363
681,-8.7597,-35.1047
682,5.5577,95.3222
686,8.9290,-75.0271


In [57]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.75)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))